In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import warnings
import os
import time
import math
import statistics as stats
import pandas as pd
from google.colab import files

In [2]:
def plot(digraph):
    
    plt.rcParams["figure.figsize"] = [12, 12]
    plt.rcParams["figure.autolayout"] = True
    
    pos = nx.circular_layout(digraph)
   
    plt.show()
    warnings.filterwarnings("ignore")
    nx.draw(digraph, pos,with_labels=True,node_color='green', font_color='white')

In [3]:
# returns true if a path exists between u and v
def existPath(matriceAdj, u, v):
   
    n = len(matriceAdj) 
    file = []
    visites = [False] * n
    file.append(u)
    while file:
        courant = file.pop(0)
        visites[courant] = True
        
        for i in range(n):
            if matriceAdj[courant,i] > 0 and visites[i] == False:
                file.append(i)
                visites[i] = True
            elif matriceAdj[courant,i] > 0 and i == v:
                return True 
    return False

In [4]:
# returns the number of nodes in at least one cycle in the given state
def nbNodesInAtLeastOneCycle(state):
    count = 0
    if (len(state) == 0):
      return 0
    adjacencyMatrix=nx.adjacency_matrix(state)
    adjacencyMatrix=adjacencyMatrix.todense()
    
   
    for node in range(0,len(adjacencyMatrix)):
      if existPath(adjacencyMatrix, node, node):
        count+=1
    return count

In [5]:
# returns a random index of an array of probabilities
def randomChoice(probaVector, sumProba):
        threshold =  np.random.uniform(0,1) * sumProba
        cumul = 0
        for i in range(len(probaVector)):
            cumul += probaVector[i]
            if (threshold < cumul):
                return i
        return len(probaVector)-1

In [6]:
# creates and returns a list of couples for reproduction from population
# individuals are chosen randomly and the probability to chose an individual from the population is proportional to its fitness
def randomSelection(population):
    couples = []
    probaVector = []
    sumProba = 0
    
    # here, we want to favor individuals with small fitness, thus we take a decreasing function of it
    for elem in population:
        fitness = math.exp(-1/(nbNodesInAtLeastOneCycle(elem)+1)) *100
        probaVector.append(fitness)
        sumProba += fitness
    print("probaVector :", probaVector)
    
    for i in range(len(population)//2):
        motherIndex = randomChoice(probaVector, sumProba)
        while(1):
            fatherIndex = randomChoice(probaVector, sumProba)
            if (motherIndex != fatherIndex):
                break
       
        mother = population[motherIndex]
        father = population[fatherIndex]
        couple = [mother, father]
        couples.append(couple)
    
    return couples

In [7]:
# takes as an argument a list of couples and creates two children out of each couple
def generateChildByCrossover(parentCouplesList,initialStateCopy ):
    newPopulation = []
    allNodes = list(initialStateCopy.nodes())
    initialEdges = list(initialStateCopy.edges())
    
    for couple in parentCouplesList :
        mother = couple[0]
        father = couple[1]
        child1 = nx.DiGraph()
        child1Nodes = []
        child2 = nx.DiGraph()
        child2Nodes = []
        
        for node in allNodes:
            motherOrFather = np.random.uniform(0,1)
    
            #child 1 --> father
            #child 2 --> mother
            if (motherOrFather > 0.5):
                if (father.has_node(node)):
                    child1Nodes.append(node)
                if (mother.has_node(node)):
                    child2Nodes.append(node)
        
            #child 1 --> mother
            #child 2 --> father
            else:
                if (mother.has_node(node)):
                    child1Nodes.append(node)
                if (father.has_node(node)):
                    child2Nodes.append(node)
                
        edgesToAddChild1 = [elem for elem in initialEdges if (elem[0] in child1Nodes and elem[1] in child1Nodes)]
        edgesToAddChild2 = [elem for elem in initialEdges if (elem[0] in child2Nodes and elem[1] in child2Nodes)]
        child1.add_edges_from(edgesToAddChild1)
        child2.add_edges_from(edgesToAddChild2)
       
        newPopulation.append(child1)
        newPopulation.append(child2)
    
    return newPopulation

In [8]:
# returns a mutated child (a node removed or added)
def mutation(state, initialStateCopy):
    stateNodes = list(state.nodes())
    stateCopy = state.copy()
    
    addOrRemoveNode = np.random.uniform(0,1)
   
    if ((addOrRemoveNode < 0.75 or initialStateCopy.number_of_nodes() == state.number_of_nodes()) and state.number_of_nodes() > 0):
        nodeToRemove = random.choice(stateNodes)
        stateCopy.remove_node(nodeToRemove)
        #print("\nnode removed : ", nodeToRemove)
   
    else:
        initialNodes = list(initialStateCopy.nodes())
        initialEdges = list(initialStateCopy.edges())
        nodeToAdd = random.choice([elem for elem in initialNodes if elem not in stateNodes])
        edgesToAdd = [elem for elem in initialEdges if (elem[0] == nodeToAdd and elem[1] in stateNodes) or (elem[1] == nodeToAdd and elem[0] in stateNodes)]
        stateCopy.add_edges_from(edgesToAdd)
        #print("\nnode added : ", nodeToAdd)
   
    return stateCopy

In [9]:
# returns the best individual and its fitness
def bestIndividual(population):
    bestIndividual = population[0]
    bestFitness = nbNodesInAtLeastOneCycle(bestIndividual)
   
    for elem in population:
        fitness = nbNodesInAtLeastOneCycle(elem)
        if(fitness < bestFitness):
            bestIndividual = elem
            bestFitness = fitness
    return (bestIndividual, bestFitness)

In [10]:
def genetic(nbNodes, probaErdosRenyi, plotResults, proportion):
  startTime = time.time()

  
  
  initialState = nx.generators.random_graphs.erdos_renyi_graph(nbNodes,probaErdosRenyi, directed= True)
  initialStateCopy = initialState.copy()

  print("We start with this initial graph : " , list(initialStateCopy.edges()))

  # we start with a population partially composed of initial state graphs
  population = [initialState] * int(len(initialState)*proportion)
  # and we complete it with a certain propotion of individuals derived from the initial state
  for i in range(int((1-proportion)*len(initialState))):
      reducedInitialState = initialState.copy()
      nodesToRemove = []
      nbNodesToRemove = random.choice([*range(len(initialState)//4, 3*len(initialState)//4)])
      for j in range(nbNodesToRemove):
        nodesToRemove.append(random.choice([elem for elem in list(initialState.nodes()) if elem not in nodesToRemove]))
      reducedInitialState.remove_nodes_from(nodesToRemove)
      population.append(reducedInitialState)
      
  fitIndividualFound = (nbNodesInAtLeastOneCycle(initialState) == 0)
  fittestIndividualInPopulation = initialState
  itNb = 0

  while not(fitIndividualFound):
      newPopulation = []
      
      # find best couples for reproduction
      parentCouplesList = randomSelection(population)

      #reproduction - crossover
      newPopulation = generateChildByCrossover(parentCouplesList, initialStateCopy)
          
      # random mutations
      for i in range(len(newPopulation)):
          probaOfMutation = np.random.uniform(0,0.3)
          if(probaOfMutation < 0.15):
              newPopulation[i] = mutation(newPopulation[i], initialStateCopy)
          
      population = newPopulation

      fittestIndividualAndFitnessInPopulation = bestIndividual(population)
      fittestIndividualInPopulation = fittestIndividualAndFitnessInPopulation[0]
      bestFitness = fittestIndividualAndFitnessInPopulation[1]
      fitIndividualFound = (bestFitness == 0 or itNb > 1000)
      itNb+=1;

  print("\n\n-------------------END OF ALGORITHM-------------------\n")
  try:
    print("Cycle in solution ? ",nx.find_cycle(fittestIndividualInPopulation))
    remainingCycles = True
  except nx.exception.NetworkXNoCycle as e:
    print("Found no cycle ")
    remainingCycles = False

  # we made an acyclic graph out of the initial graph
  # now we must deduce the cycle cutter and display it
  cycleCutter = [elem for elem in list(initialStateCopy.nodes()) if elem not in list(fittestIndividualInPopulation.nodes())]

  finishTime = time.time()
  executionTime = finishTime - startTime

  print("\n\n-------------------CONCLUSION-------------------\n\nThe initial graph had", nbNodes, "nodes and", nbNodesInAtLeastOneCycle(initialStateCopy), "nodes in at least one cycle.")
  print("\nAnd we obtained the following acyclic graph :\n\n" , list(fittestIndividualInPopulation.edges()))
  print("\nThus the cycle-cutter is :", cycleCutter, ", the genetic algorithm removed", len(cycleCutter), "nodes, with execution time =", round(executionTime,2), "seconds.")

  if (plotResults):
    print("\n\n-------------------INITIAL STATE (Fig 1) VS FINAL STATE (Fig 2)-------------------")
    plot(initialStateCopy)
    plot(fittestIndividualInPopulation)
  return (len(cycleCutter), executionTime, remainingCycles)

In [ ]:
#Testez l’algorithme génétique dans cette cellule **********************************************************************************
genetic(30, 0.1, True, 0.2)

In [ ]:
# statistics
'''
nbOfNodesToConsider = [10,30,70,100]
probasErdosRenyiToConsider = [0.02,0.04,0.06,0.08,0.1,0.2,0.3,0.4,0.5]
proportion=[0.1,0.2,0.5,0.7]

sampleSize = 15

for nbNode in nbOfNodesToConsider: 
 
  nodeListDf = []
  probaListDf = []
  proportionListDf = []
  meanCycleCutterListDf = []
  standardDeviationCycleCutterListDf = []
  meanExecutionTimeListDf = []
  standarDeviationExecutionTimeListDf = []
  for p in proportion:
    
    for proba in probasErdosRenyiToConsider:
        nbNodes = nbNode
        probaErdosRenyi = proba
        
        # plotResults = True only if trial for one graph, not in a loop for several graphs

        resultList = [0] * sampleSize
        executionTimeList = [0] * sampleSize
        #generating 50 graphs with these parameters
        for i in range(sampleSize):
          result = genetic(nbNodes,probaErdosRenyi,False,p)
          resultList[i] = result[0]
          executionTimeList[i] = result[1]

        averageExecutionTime = round((sum(executionTimeList)/len(executionTimeList)),2)
        mean = round(sum(resultList)/len(resultList),2)
        standardDeviation = round(math.sqrt(stats.pvariance(resultList)),2)
        standardDeviationExecutionTime = round(math.sqrt(stats.pvariance(executionTimeList)),2)

        print("\n\n-------------------MEAN & STANDARD DEVIATION-------------------\n")
        print("The initial graph had", nbNodes, "nodes and probability of Erdos-Renyi", probaErdosRenyi,".")
        print("\nIn average, the cycle-cutter contains :", mean, "nodes, with a standard deviation of :", standardDeviation,".")
        print("\nIn average, the execution time of the hill climbing algorithm with the above parameters is :", averageExecutionTime, "seconds, with a standard deviation of :",standardDeviationExecutionTime,".")

        #completing lists for the dataframe
        nodeListDf.append(nbNode)
        probaListDf.append(proba)
        proportionListDf.append(p)
        meanCycleCutterListDf.append(mean)
        standardDeviationCycleCutterListDf.append(standardDeviation)
        meanExecutionTimeListDf.append(averageExecutionTime)
        standarDeviationExecutionTimeListDf.append(standardDeviationExecutionTime)

  #creating a report for each number of nodes in graph
  results = {"nombre de sommets" : nodeListDf, "proba Erdos-Renyi" : probaListDf,"proportion d'individus initiaux dans la population initiale":proportionListDf ,"moyenne taille coupe cycle" : meanCycleCutterListDf, "écart-type taille coupe cycle" : standardDeviationCycleCutterListDf, "moyenne temps exécution" : meanExecutionTimeListDf, "écart-type temps exécution" : standarDeviationExecutionTimeListDf,}
  results_df = pd.DataFrame(results)
  moncsv = results_df.to_csv("results_genetic_" + str(nbNode) +".csv", sep=";")
  files.download("results_genetic_" + str(nbNode) +".csv") 
  '''